<a href="https://colab.research.google.com/github/LeonardoGomesG/Simula-o-Covid/blob/main/CovidSimulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
from scipy.stats import norm
import random
import time
import array
import matplotlib.pyplot as plt
import numpy as np

In [124]:
# No measures
all_data = {
  "no_counter_measures": {
      "population": 5000,
      "startingImmunity%": 0,
      "startingInfecters": 1,
      "daysContagious": 7,
      "lockdownDays": 0,
      "maskDays": 100,
  },

  # Lockdown 15 days
  "lockdown_20": { 
      "population": 5000,
      "startingImmunity%": 0,
      "startingInfecters": 1,
      "daysContagious": 7,
      "lockdownDays": 20,
      "maskDays": 100,
  },

  # Lockdown 20 days
  "lockdown_25": {
      "population": 5000,
      "startingImmunity%": 0,
      "startingInfecters": 1,
      "daysContagious": 7,
      "lockdownDays": 25,
      "maskDays": 100,
  },

  # Masks 15 days
  "masks_15": {
      "population": 5000,
      "startingImmunity%": 0,
      "startingInfecters": 1,
      "daysContagious": 7,
      "lockdownDays": 0,
      "maskDays": 15,
  },
}


In [111]:
peopleDictionary = []
#simulation of a single person

class Person():
    def __init__(self, startingImmunity):
        if random.randint(0,100)<startingImmunity:
            self.immunity = True
        else:
            self.immunity = False
        self.contagiousness = 0
        self.mask = False
        self.contagiousDays = 0
        #use gaussian distribution for number of friends; average is 5 friends
        self.friends = int((norm.rvs(size=1,loc=0.5,scale=0.15)[0]*10).round(0))
    def wearMask(self):
        self.contagiousness /= 2
        
def initiateSim(data):
    numPeople = data["population"]
    startingImmunity = data["startingImmunity%"]
    startingInfecters = data["startingInfecters"]
#     numPeople = int(input("Population: "))
#     startingImmunity = int(input("Percentage of people with natural immunity: "))
#     startingInfecters = int(input("How many people will be infectious at t=0: "))
    for x in range(0,numPeople):
        peopleDictionary.append(Person(startingImmunity))
    for x in range(0,startingInfecters):
        peopleDictionary[random.randint(0,len(peopleDictionary)-1)].contagiousness = int((norm.rvs(size=1,loc=0.5,scale=0.15)[0]*10).round(0)*10)
    
    # daysContagious = data["daysContagious"]
    # lockdownDay = data["lockdownDays"]
    # maskDay = data["maskDays"]
    # daysContagious = int(input("How many days contagious: "))
    # lockdownDay = int(input("Day for lockdown to be enforced: "))
    # maskDay = int(input("Day for masks to be used: "))

def runDay(daysContagious, lockdown):
    #this section simulates the spread, so it only operates on contagious people, thus:
    for person in [person for person in peopleDictionary if person.contagiousness>0 and person.friends>0]:
        peopleCouldMeetToday = int(person.friends/2)
        if peopleCouldMeetToday > 0:
            peopleMetToday = random.randint(0,peopleCouldMeetToday)
        else:
            peopleMetToday = 0
            
        if lockdown == True:
            peopleMetToday= 0
            
        for x in range(0,peopleMetToday):
            friendInQuestion = peopleDictionary[random.randint(0,len(peopleDictionary)-1)]
            if random.randint(0,100)<person.contagiousness and friendInQuestion.contagiousness == 0 and friendInQuestion.immunity==False:
                friendInQuestion.contagiousness = int((norm.rvs(size=1,loc=0.5,scale=0.15)[0]*10).round(0)*10)
                # print(peopleDictionary.index(person), " >>> ", peopleDictionary.index(friendInQuestion))
            
    for person in [person for person in peopleDictionary if person.contagiousness>0]:
        person.contagiousDays += 1
        if person.contagiousDays > daysContagious:
            person.immunity = True
            person.contagiousness = 0
            # print("|||healed:", peopleDictionary.index(person), " |||")


def get_infected_num(data):
  infected = []
  infected.clear()
  infected.append(data["startingInfecters"])
  # print("DAY 0:", data["startingInfecters"])
  lockdown = False
  flag = False
  for x in range(1,50):    
    if x==data["lockdownDays"]:
        lockdown = True
        
    if x == data["maskDays"]:
        for person in peopleDictionary:
            person.wearMask()
            
    runDay(data["daysContagious"],lockdown)
    infected_num = len([person for person in peopleDictionary if person.contagiousness>0])
    # print(f"DAY {x}: {infected_num}")
    infected.append(infected_num)
    # write = str(infected_num) + "\n"
    # saveFile.write(write)
    # print(len([person for person in peopleDictionary if person.contagiousness>0]), " people are contagious on this day.")

  return infected

def get_stabilization90_day(infected, population):
  flag = False
  day=0
  for num in infected:
    day+=1
    if not flag and num > population*0.1:
      flag = True
    if flag and num < population*0.1:
      print("stabilized at ", day)
      return day
  
  if flag:
    print("never stable")
    return 100000000000000
  else: 
    print("always stable")
    return 0


# def plot_graphs(data):
#   lockdown = False
#   daysContagious, lockdownDay, maskDay = initiateSim()
#   infected = []

#   for data_key, data_value in all_data:
#     infected = get_infected_for_data(data)
#     days = [...infected.keys()]
#     plt.(days, infected, label=str(data_key))

#   plt.xlabel('days')
#   plt.ylabel('n_infected')
#   plt.title("Covid Transmission")


            

In [160]:
  lockdown = False
  infected = []  
  stabilized = {}
  for data_key, _ in all_data.items():
    stabilized[str(data_key)] = []

  for i in range(1,20):
    for data_key, data_value in all_data.items():
      print(f"---------------------- {data_key} ----------------------")
      peopleDictionary.clear()
      initiateSim(data_value)
      infected = get_infected_num(data_value)
      # stabilized[str(data_key)] = []
      stabilization_day = get_stabilization90_day(infected, data_value["population"])
      stabilized[str(data_key)].append(stabilization_day)
      days = list(range(0, 50))
      # print(f"days: {days}\ninfected: {infected}")
      # plt.plot(days, infected, label=str(data_key))

  # plt.xlabel('days')
  # plt.ylabel('n_infected')
  # plt.title("Covid Transmission")
  # plt.legend()

  print("stabilized:\n", stabilized)

---------------------- no_counter_measures ----------------------
stabilized at  35
---------------------- lockdown_15 ----------------------
stabilized at  27
---------------------- lockdown_20 ----------------------
stabilized at  32
---------------------- masks_15 ----------------------
stabilized at  34
---------------------- no_counter_measures ----------------------
stabilized at  33
---------------------- lockdown_15 ----------------------
stabilized at  27
---------------------- lockdown_20 ----------------------
stabilized at  32
---------------------- masks_15 ----------------------
stabilized at  32
---------------------- no_counter_measures ----------------------
stabilized at  35
---------------------- lockdown_15 ----------------------
stabilized at  27
---------------------- lockdown_20 ----------------------
stabilized at  30
---------------------- masks_15 ----------------------
stabilized at  36
---------------------- no_counter_measures ----------------------
stabili

In [165]:
import statistics
import pprint

metrics = {}
metrics.clear()
for key, value in stabilized.items():
  metrics[key] = {
      "mean": statistics.mean(value),
      "standard_deviation": statistics.stdev(value),
      "variance": statistics.variance(value),
  }

print("metrics:\n")
pprint.pprint(metrics)

metrics_without_outliers = {}
clean_value = []
for key, value in stabilized.items():
  mean = statistics.mean(value)
  i=0
  clean_value.clear()
  for days in value:
    if days!=0:
      clean_value.append(value[i])

    i+=1

  
  metrics_without_outliers[key] = {
      "mean": statistics.mean(clean_value),
      "standard_deviation": statistics.stdev(clean_value),
      "variance": statistics.variance(clean_value),
  }

print("metrics without outliers:\n")
pprint.pprint(metrics_without_outliers)



metrics:

{'lockdown_15': {'mean': 25.105263157894736,
                 'standard_deviation': 6.109143755625708,
                 'variance': 37.32163742690059},
 'lockdown_20': {'mean': 27.57894736842105,
                 'standard_deviation': 9.777046756298846,
                 'variance': 95.5906432748538},
 'masks_15': {'mean': 33.578947368421055,
              'standard_deviation': 8.636587478561992,
              'variance': 74.5906432748538},
 'no_counter_measures': {'mean': 33.78947368421053,
                         'standard_deviation': 3.4250538903857826,
                         'variance': 11.730994152046783}}
metrics without outliers:

{'lockdown_15': {'mean': 26.5,
                 'standard_deviation': 0.6183469424008423,
                 'variance': 0.38235294117647056},
 'lockdown_20': {'mean': 30.823529411764707,
                 'standard_deviation': 1.1311108542958477,
                 'variance': 1.2794117647058822},
 'masks_15': {'mean': 35.44444444444444,
      